# Nonparametric ML Models - Cumulative Lab

## Introduction

In this cumulative lab, you will apply two nonparametric models you have just learned — k-nearest neighbors and decision trees — to the forest cover dataset.

## Objectives

* Practice identifying and applying appropriate preprocessing steps
* Perform an iterative modeling process, starting from a baseline model
* Explore multiple model algorithms, and tune their hyperparameters
* Practice choosing a final model across multiple model algorithms and evaluating its performance

## Your Task: Complete an End-to-End ML Process with Nonparametric Models on the Forest Cover Dataset

![line of pine trees](https://curriculum-content.s3.amazonaws.com/data-science/images/trees.jpg)

Photo by <a href="https://unsplash.com/@michaelbenz?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Michael Benz</a> on <a href="/s/photos/forest?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>

### Business and Data Understanding

To repeat the previous description:

> Here we will be using an adapted version of the forest cover dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/covertype). Each record represents a 30 x 30 meter cell of land within Roosevelt National Forest in northern Colorado, which has been labeled as `Cover_Type` 1 for "Cottonwood/Willow" and `Cover_Type` 0 for "Ponderosa Pine". (The original dataset contained 7 cover types but we have simplified it.)

The task is to predict the `Cover_Type` based on the available cartographic variables:

In [1]:
# Run this cell without changes
import pandas as pd

df = pd.read_csv('data/forest_cover.csv')
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Cover_Type
0,2553,235,17,351,95,780,188,253,199,1410,...,0,0,0,0,0,0,0,0,0,0
1,2011,344,17,313,29,404,183,211,164,300,...,0,0,0,0,0,0,0,0,0,0
2,2022,24,13,391,42,509,212,212,134,421,...,0,0,0,0,0,0,0,0,0,0
3,2038,50,17,408,71,474,226,200,102,283,...,0,0,0,0,0,0,0,0,0,0
4,2018,341,27,351,34,390,152,188,168,190,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38496,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,0
38497,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,0
38498,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,0
38499,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,0


> As you can see, we have over 38,000 rows, each with 52 feature columns and 1 target column:

> * `Elevation`: Elevation in meters
> * `Aspect`: Aspect in degrees azimuth
> * `Slope`: Slope in degrees
> * `Horizontal_Distance_To_Hydrology`: Horizontal dist to nearest surface water features in meters
> * `Vertical_Distance_To_Hydrology`: Vertical dist to nearest surface water features in meters
> * `Horizontal_Distance_To_Roadways`: Horizontal dist to nearest roadway in meters
> * `Hillshade_9am`: Hillshade index at 9am, summer solstice
> * `Hillshade_Noon`: Hillshade index at noon, summer solstice
> * `Hillshade_3pm`: Hillshade index at 3pm, summer solstice
> * `Horizontal_Distance_To_Fire_Points`: Horizontal dist to nearest wildfire ignition points, meters
> * `Wilderness_Area_x`: Wilderness area designation (3 columns)
> * `Soil_Type_x`: Soil Type designation (39 columns)
> * `Cover_Type`: 1 for cottonwood/willow, 0 for ponderosa pine

This is also an imbalanced dataset, since cottonwood/willow trees are relatively rare in this forest:

In [2]:
# Run this cell without changes
print("Raw Counts")
print(df["Cover_Type"].value_counts())
print()
print("Percentages")
print(df["Cover_Type"].value_counts(normalize=True))

Raw Counts
Cover_Type
0    35754
1     2747
Name: count, dtype: int64

Percentages
Cover_Type
0    0.928651
1    0.071349
Name: proportion, dtype: float64


Thus, a baseline model that always chose the majority class would have an accuracy of over 92%. Therefore we will want to report additional metrics at the end.

### Previous Best Model

In a previous lab, we used SMOTE to create additional synthetic data, then tuned the hyperparameters of a logistic regression model to get the following final model metrics:

* **Log loss:** 0.13031294393913376
* **Accuracy:** 0.9456679825472678
* **Precision:** 0.6659919028340081
* **Recall:** 0.47889374090247455

In this lab, you will try to beat those scores using more-complex, nonparametric models.

### Modeling

Although you may be aware of some additional model algorithms available from scikit-learn, for this lab you will be focusing on two of them: k-nearest neighbors and decision trees. Here are some reminders about these models:

#### kNN - [documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

This algorithm — unlike linear models or tree-based models — does not emphasize learning the relationship between the features and the target. Instead, for a given test record, it finds the most similar records in the training set and returns an average of their target values.

* **Training speed:** Fast. In theory it's just saving the training data for later, although the scikit-learn implementation has some additional logic "under the hood" to make prediction faster.
* **Prediction speed:** Very slow. The model has to look at every record in the training set to find the k closest to the new record.
* **Requires scaling:** Yes. The algorithm to find the nearest records is distance-based, so it matters that distances are all on the same scale.
* **Key hyperparameters:** `n_neighbors` (how many nearest neighbors to find; too few neighbors leads to overfitting, too many leads to underfitting), `p` and `metric` (what kind of distance to use in defining "nearest" neighbors)

#### Decision Trees - [documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

Similar to linear models (and unlike kNN), this algorithm emphasizes learning the relationship between the features and the target. However, unlike a linear model that tries to find linear relationships between each of the features and the target, decision trees look for ways to split the data based on features to decrease the entropy of the target in each split.

* **Training speed:** Slow. The model is considering splits based on as many as all of the available features, and it can split on the same feature multiple times. This requires exponential computational time that increases based on the number of columns as well as the number of rows.
* **Prediction speed:** Medium fast. Producing a prediction with a decision tree means applying several conditional statements, which is slower than something like logistic regression but faster than kNN.
* **Requires scaling:** No. This model is not distance-based. You also can use a `LabelEncoder` rather than `OneHotEncoder` for categorical data, since this algorithm doesn't necessarily assume that the distance between `1` and `2` is the same as the distance between `2` and `3`.
* **Key hyperparameters:** Many features relating to "pruning" the tree. By default they are set so the tree can overfit, and by setting them higher or lower (depending on the hyperparameter) you can reduce overfitting, but too much will lead to underfitting. These are: `max_depth`, `min_samples_split`, `min_samples_leaf`, `min_weight_fraction_leaf`, `max_features`, `max_leaf_nodes`, and `min_impurity_decrease`. You can also try changing the `criterion` to "entropy" or the `splitter` to "random" if you want to change the splitting logic.

### Requirements

#### 1. Prepare the Data for Modeling

#### 2. Build a Baseline kNN Model

#### 3. Build Iterative Models to Find the Best kNN Model

#### 4. Build a Baseline Decision Tree Model

#### 5. Build Iterative Models to Find the Best Decision Tree Model

#### 6. Choose and Evaluate an Overall Best Model

## 1. Prepare the Data for Modeling

The target is `Cover_Type`. In the cell below, split `df` into `X` and `y`, then perform a train-test split with `random_state=42` and `stratify=y` to create variables with the standard `X_train`, `X_test`, `y_train`, `y_test` names.

Include the relevant imports as you go.

In [5]:
# Your code here
from sklearn.model_selection import train_test_split

X= df.drop('Cover_Type', axis=1)
y= df['Cover_Type']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

Now, instantiate a `StandardScaler`, fit it on `X_train`, and create new variables `X_train_scaled` and `X_test_scaled` containing values transformed with the scaler.

In [6]:
# Your code here
from sklearn.preprocessing import StandardScaler

scaler= StandardScaler()

X_train_scaled= scaler.fit_transform(X_train)
X_test_scaled= scaler.transform(X_test)

The following code checks that everything is set up correctly:

In [7]:
# Run this cell without changes

# Checking that df was separated into correct X and y
assert type(X) == pd.DataFrame and X.shape == (38501, 52)
assert type(y) == pd.Series and y.shape == (38501,)

# Checking the train-test split
assert type(X_train) == pd.DataFrame and X_train.shape == (28875, 52)
assert type(X_test) == pd.DataFrame and X_test.shape == (9626, 52)
assert type(y_train) == pd.Series and y_train.shape == (28875,)
assert type(y_test) == pd.Series and y_test.shape == (9626,)

# Checking the scaling
assert X_train_scaled.shape == X_train.shape
assert round(X_train_scaled[0][0], 3) == -0.636
assert X_test_scaled.shape == X_test.shape
assert round(X_test_scaled[0][0], 3) == -1.370

## 2. Build a Baseline kNN Model

Build a scikit-learn kNN model with default hyperparameters. Then use `cross_val_score` with `scoring="neg_log_loss"` to find the mean log loss for this model (passing in `X_train_scaled` and `y_train` to `cross_val_score`). You'll need to find the mean of the cross-validated scores, and negate the value (either put a `-` at the beginning or multiply by `-1`) so that your answer is a log loss rather than a negative log loss.

Call the resulting score `knn_baseline_log_loss`.

Your code might take a minute or more to run.

In [19]:
# Replace None with appropriate code

# Relevant imports
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Creating the model
knn_baseline_model = KNeighborsClassifier()

# Perform cross-validation
knn_baseline_log_loss = -cross_val_score(knn_baseline_model, 
                                         X_train_scaled, 
                                         y_train, 
                                         scoring="neg_log_loss").mean()


knn_baseline_log_loss

0.12964546386734577

Our best logistic regression model had a log loss of 0.13031294393913376

Is this model better? Compare it in terms of metrics and speed.

In [10]:
# Replace None with appropriate text
"""
The log loss is lower on the baseline KNN model than the previous model, 
pointing to this KNN baseline model being the better model.

In terms of speed, both the results for this model and the previous model 
were almost instant on my computer. However, on a less powerful computer, 
the KNN model could inherently take longer than the previous model to run. 
"""

'\nThe log loss is lower on the baseline KNN model than the previous model, \npointing to this KNN baseline model being the better model.\n\nIn terms of speed, both the results for this model and the previous model \nwere almost instant on my computer. However, on a less powerful computer, \nthe KNN model could inherently take longer than the previous model to run. \n'

## 3. Build Iterative Models to Find the Best kNN Model

Build and evaluate at least two more kNN models to find the best one. Explain why you are changing the hyperparameters you are changing as you go. These models will be *slow* to run, so be thinking about what you might try next as you run them.

In [15]:
# Your code here (add more cells as needed)
from sklearn.model_selection import GridSearchCV

"""
parameters to tune

n_neighbors: How many neighbors the model looks at when making a decision. 
Smaller numbers are more sensitive, while larger numbers allow the model to generalize more.
The prediction will be made based on the majority of these N neighbors. 

metric: Defines how the model determines the distance of the point in question to
its neighbors. Different methods of distance calculation affect which data 
points will and won't be included in the n_neighbors taken into account. 

p: If the 'metric' is set to Minkowski, this modifies how that distance
is computed.

weights: Changes how much sway the different n_neighbors have in
the prediction decision. If uniform, all have equal sway. If distance,
the further the data point the less sway it has. 
"""

knn = KNeighborsClassifier()

param_grid = {'n_neighbors': [5, 50, 100, 200], 
              'p': [1, 2, 'l_p'], 
              'metric': ['minkowski', 'euclidean', 'manhattan'],
              'weights': ['uniform', 'distance']}

gs_knn = GridSearchCV(knn, param_grid, cv = 3, scoring='neg_log_loss', n_jobs=-1)
gs_knn.fit(X_train_scaled, y_train)

gs_knn.best_params_

/opt/anaconda3/envs/my_py_3_8_5_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
72 fits failed out of a total of 216.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
72 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/my_py_3_8_5_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/envs/my_py_3_8_5_env/lib/python3.8/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/opt/anaconda3/envs/my_py_3_8_5_env/lib/python3.8/site-packages/sklearn/base.py", line 638, in _validate_params
  

{'metric': 'minkowski', 'n_neighbors': 50, 'p': 1, 'weights': 'distance'}

In [16]:
# Your code here (add more cells as needed)
"""
Based on the above cross-validation grid search, the best hyper parameter
options to use for a KNN minimized log loss are the follow. Please note this is 
only out of the values I let it test and not all possible values.
"""

'\nBased off of the above cross validation grid search, the hyper parameter\noptions to use for a KNN minimized log loss are the follow. Please note this is \nonly out of the values I let it test and not all possible values.\n'

In [17]:
# Your code here (add more cells as needed)
# Creating the model
knn_tuned = KNeighborsClassifier(metric= 'minkowski',
                                n_neighbors= 50,
                                p= 1,
                                weights= 'distance')

# Perform cross-validation
knn_tuned_log_loss = -cross_val_score(knn_tuned, 
                                         X_train_scaled, 
                                         y_train, 
                                         scoring="neg_log_loss").mean()


knn_tuned_log_loss

0.06628925339389122

In [20]:
"""
This tuned KNN model cuts the baseline KNN model's log loss score in half,
making the tuned KNN model the current best model.
"""

"\nThis tuned KNN model cuts the baseline KNN model's log loss score in half,\nmaking the tuned KNN model the current best model.\n"

## 4. Build a Baseline Decision Tree Model

Now that you have chosen your best kNN model, start investigating decision tree models. First, build and evaluate a baseline decision tree model, using default hyperparameters (with the exception of `random_state=42` for reproducibility).

(Use cross-validated log loss, just like with the previous models.)

In [21]:
# Your code here

from sklearn.tree import DecisionTreeClassifier

dt_baseline= DecisionTreeClassifier(random_state= 42)

dt_baseline_log_loss = -cross_val_score(dt_baseline, 
                                         X_train, 
                                         y_train, 
                                         scoring="neg_log_loss").mean()


dt_baseline_log_loss

0.7352281158853683

Interpret this score. How does this compare to the log loss from our best logistic regression and best kNN models? Any guesses about why?

In [22]:
# Replace None with appropriate text
"""
This baseline decision tree's log loss score is much higher than the KNN and logistic
regression model scores. This could be due to the overfitting on the training data. 
We will see if tuning hyper parameters can fix that. 
"""

"\nThis baseline decision tree's log loss score is much higher than the KNN and logistic\nregression model scores. This could be due to the overfitting on the training data. \nWe will see if tuning hyper parameters can fix that. \n"

## 5. Build Iterative Models to Find the Best Decision Tree Model

Build and evaluate at least two more decision tree models to find the best one. Explain why you are changing the hyperparameters you are changing as you go.

In [23]:
# Your code here (add more cells as needed)
"""
parameters to tune

max_depth: How deep the tree can go or how many questions (splits) the decision tree can ask about 
the data before it makes a decision. With more splits you learn more about the data, but can risk overfitting. 

min_samples_split: This dictates the minimum number of data points a question line must have left in order
for it to ask another question (about the data). The lower the number the easier it is to ask more questions,
which can lead to overfitting. 

min_samples_leaf: Allowing only higher values helps counteract overfitting while allowing lower values helps 
pick up on more patterns. 

min_weight_fraction_leaf: Similar to min samples leaf Allowing only higher values helps counteract 
overfitting while allowing lower values helps pick up on more patterns. 

max_features: This determines how many of the independent variables are allowed to be taken into 
consideration when asking questions (splitting) about the data. Here the lower the number the 
less the tree will overfit on the training data. 

max_leaf_nodes: While higher values here can tell you more about the data you're training on,
it can also lead to overfitting. 

min_impurity_decrease: This is another tradeoff between learning more about your data and overfitting
to it. 

criterion: This determines how the model decides which questions (splits) are the most effective
questions to ask in the question line. 

splitter: This chooses how each decision line question is split. Do we want more accuracy, or do we 
want to avoid more overfitting?
"""

dt = DecisionTreeClassifier(random_state= 42)

dt_param_grid = {'max_depth': [1, 3, 5, 10, 20], 
              'min_samples_split': [2, 5, 10, 20, 50], 
              'min_samples_leaf': [1, 2, 5, 10, 20],
              'min_weight_fraction_leaf': [0.0, 0.01, 0.05, 0.1],
              'max_features': [None, 'sqrt', 'log2', 3, 5],
              'max_leaf_nodes': [None, 10, 20, 50, 100, 200],
              'min_impurity_decrease': [0.0, 0.01, 0.05, 0.1],
              'criterion': ['gini', 'entropy'],
              'splitter': ['best', 'random']}

gs_dt = GridSearchCV(dt, dt_param_grid, cv = 3, scoring='neg_log_loss', n_jobs=-1)
gs_dt.fit(X_train, y_train)

gs_dt.best_params_

{'criterion': 'gini',
 'max_depth': 10,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 20,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'splitter': 'random'}

In [24]:
# Your code here (add more cells as needed)
"""
Based on the above cross-validation grid search, the best hyperparameter
options to use for a Decision Tree minimized log loss are the follow. Please note this is 
only out of the values I let it test and not all possible values.
"""

'\nBased off of the above cross validation grid search, the best hyper parameter\noptions to use for a Decision Tree minimized log loss are the follow. Please note this is \nonly out of the values I let it test and not all possible values.\n'

In [27]:
# Your code here (add more cells as needed)
dt_tuned= DecisionTreeClassifier(random_state= 42,
                             criterion= 'gini',
                             max_depth= 10,
                             max_features= None,
                             max_leaf_nodes= None,
                             min_impurity_decrease= 0.0,
                             min_samples_leaf= 20,
                             min_samples_split= 2,
                             min_weight_fraction_leaf= 0.0,
                             splitter= 'random')

dt_tuned_log_loss = -cross_val_score(dt_tuned, 
                                         X_train, 
                                         y_train, 
                                         scoring="neg_log_loss").mean()


dt_tuned_log_loss

0.09999571722556819

In [29]:
"""
Even though this tuned decision tree is a massive improvement from the baseline decision tree
it is still not more effective (in terms of lowest negative log loss) than the tuned KNN model.
It is however more effective than the logistic regression model.
"""

'\nEven though this tuned decision tree is a massive improvement from the baseline decision tree\nit is still not more effective (in terms of lowest negative log loss) than the tuned KNN model.\nIt is however more effective than the logistic regression model.\n'

## 6. Choose and Evaluate an Overall Best Model

Which model had the best performance? What type of model was it?

Instantiate a variable `final_model` using your best model with the best hyperparameters.

In [30]:
# Replace None with appropriate code

"""
The best performing model was the grid-search tuned KNN model.
"""

final_model = KNeighborsClassifier(metric= 'minkowski',
                                n_neighbors= 50,
                                p= 1,
                                weights= 'distance')

# Fit the model on the full training data
# (scaled or unscaled depending on the model)
final_model.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=50, p=1, weights='distance')

Now, evaluate the log loss, accuracy, precision, and recall. This code is mostly filled in for you, but you need to replace `None` with either `X_test` or `X_test_scaled` depending on the model you chose.

In [32]:
# Replace None with appropriate code
from sklearn.metrics import accuracy_score, precision_score, recall_score, log_loss

preds = final_model.predict(X_test_scaled)
probs = final_model.predict_proba(X_test_scaled)

print("log loss: ", log_loss(y_test, probs))
print("accuracy: ", accuracy_score(y_test, preds))
print("precision:", precision_score(y_test, preds))
print("recall:   ", recall_score(y_test, preds))

log loss:  0.0667076552276924
accuracy:  0.9766258051111573
precision: 0.9095744680851063
recall:    0.7467248908296943


Interpret your model performance. How would it perform on different kinds of tasks? How much better is it than a "dummy" model that always chooses the majority class, or the logistic regression described at the start of the lab?

In [33]:
# Replace None with appropriate text
"""
I believe that overall this model performed pretty well. 

With an accuracy score of 97.66% it performs about 5% better than the dummy model that 
always guesses each instance is not a Cottonwood/ Willow tree (~92% accuracy), and it 
is about 3% more accurate than the logistic regression model. 

The precision score tells us that 90.95% of all the trees that are classified as 
Cottonwood/ Willows actually are, and the recal score tells us that out of all 
the Cottonwood/ Willow trees this model will correctly identify around 75% of them.
This tells us that the model is more specific than sensitive, but still performs fairly
well in both areas.

Should the predictive threshold goals be modified, the model's predictions could be 
tailored to more align those requests.
"""

"\nI believe that overall this model performed pretty well. \n\nWith an accuracy score of 97.66% it performs about 5% better than the dummy model that \nalways guesses each instance is not a Cottonwood/ Willow tree (~92% accuracy), and it \nis about 3% more accurate than the logistic regression model. \n\nThe precision score tells us that 90.95% of all the trees that are classified as \nCottonwood/ Willows actually are, and the recal score tells us that out of all \nthe Cottonwood/ Willow trees this model will correctly identify around 75% of them.\nThis tells us that the model is more specific than sensitive, but still performs fairly\nwell in both areas.\n\nShould the predictive threshold goals be modified, the model's predictions could be \ntailored to more align those requests.\n"

## Conclusion

In this lab, you practiced the end-to-end machine learning process with multiple model algorithms, including tuning the hyperparameters for those different algorithms. You saw how nonparametric models can be more flexible than linear models, potentially leading to overfitting but also potentially reducing underfitting by being able to learn non-linear relationships between variables. You also likely saw how there can be a tradeoff between speed and performance, with good metrics correlating with slow speeds.